# Practical Activity 1 (**PRA1**)

## Evaluable Practical Exercise

<u>General considerations</u>:

- The proposed solution cannot use methods, functions or parameters declared **_deprecated_** in future versions.
- This activity must be carried out on a **strictly individual** basis. Any indication of copying will be penalized with a failure for all parties involved and the possible negative evaluation of the subject in its entirety.
- It is necessary for the student to indicate **all the sources** that she/he has used to carry out the PRA. If not, the student will be considered to have committed plagiarism, being penalized with a failure and the possible negative evaluation of the subject in its entirety.

<u>Delivery format</u>:

- Some exercises may require several minutes of execution, so the delivery must be done in **Notebook format** and in **HTML format**, where the code, results and comments of each exercise can be seen. You can export the notebook to HTML from the menu File $\to$ Download as $\to$ HTML.
- There is a special type of cell to hold text. This type of cell will be very useful to answer the different theoretical questions posed throughout the activity. To change the cell type to this type, in the menu: Cell $\to$ Cell Type $\to$ Markdown.

<div class="alert alert-block alert-info">
<strong>Name and surname: Victor Brao Ruiz </strong>
</div>

## Introduction

Blackjack environment is part of the Gymnasium's [Toy Text](https://gymnasium.farama.org/environments/toy_text/) environments. Blackjack is a card game where the goal is to beat the dealer by obtaining cards that sum to closer to 21 (without going over 21) than the dealer's cards.

The card values are, as depicted in the following figure:
- Face cards (Jack, Queen, King) have a point value of **10**.
- Aces can either count as **11** (called a "usable ace") or **1**.
- Numerical cards (**2-9**) have a value equal to their number.

<img src="./figs/BlackJackCards.png" />

Game Dynamics:
1. The game starts with the dealer having one face up and one face down card, while the player has two face up cards. All cards are drawn from an infinite deck (i.e. with replacement).
2. The player has a total sum of cards. They can request additional cards (**hit**) until they decide to stop (**stick**) or exceed 21 (**bust**), which results in an immediate loss.
3. After the player decides to stick, the dealer reveals their face-down card and draws cards until their total is 17 or greater. If the dealer goes bust, the player wins.
4. If neither the player nor the dealer goes bust, the winner is whoever has a sum closer to 21.

Further information could be found at:
- Gymnasium [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/)

In order to initialize the environment, we will use `natural=True` to give an additional reward for starting with a natural blackjack, i.e. starting with an ace and ten (sum is 21), as depicted in the following piece of code:

In [15]:
import gymnasium as gym

env = gym.make('Blackjack-v1', natural=True, sab=False)

In [16]:
print("Action space is {} ".format(env.action_space))
print("Observation space is {} ".format(env.observation_space))

Action space is Discrete(2) 
Observation space is Tuple(Discrete(32), Discrete(11), Discrete(2)) 


## Part 1. Naïve Policy

Implement an agent that carries out the following deterministic policy: 
- The agent will **stick** if it gets a score of 20 or 21.
- Otherwise, it will **hit**.

<u>Questions</u> (**1 point**): 
1. Using this agent, simulate 100,000 games and calculate the agent's return (total accumulated reward).
2. Additionally, calculate the % of wins, natural wins, losses and draws. 
3. Comment on the results.

In [17]:
import numpy as np

def naive_policy(player_sum):
    return 0 if player_sum >= 20 else 1  # 0=stick, 1=hit


n_episodes = 100000
total_reward = 0
wins = 0
natural_wins = 0
losses = 0
draws = 0

for i in range(n_episodes):
    state, info = env.reset()
    terminated = False
    truncated = False
    while not (terminated or truncated):
        player_sum = state[0]
        action = naive_policy(player_sum)
        state, reward, terminated, truncated, info = env.step(action)
    total_reward += reward
    if reward == 1.0:
        wins += 1
    elif reward == 1.5:
        natural_wins += 1
        wins += 1
    elif reward == 0.0:
        draws += 1
    elif reward == -1.0:
        losses += 1

print(f"Total accumulated reward: {total_reward}")
print(f"Win percentage: {wins / n_episodes * 100:.2f}%")
print(f"Natural win percentage: {natural_wins / n_episodes * 100:.2f}%")
print(f"Loss percentage: {losses / n_episodes * 100:.2f}%")
print(f"Draw percentage: {draws / n_episodes * 100:.2f}%")

Total accumulated reward: -32916.5
Win percentage: 29.54%
Natural win percentage: 4.14%
Loss percentage: 64.52%
Draw percentage: 5.94%


With this first basic policy implementation we have a loss percentage of almost 65%, with a win % of 29.43%, which also includes the % of natural wins (a 4.13%). As far as I know, the % of natural wins is the only one that souldn't change much in the future, as the probabilities of a natural blackjack are independent of the policy chosen. The results seem pretty consisistent with the policy we have implemented. Of course, the accumulated reward is just a reflection of this low win rate.

## Part 2. Monte Carlo method

The objective of this section is to estimate the optimal policy using Monte Carlo methods. Specifically, you can choose and implement one of the algorithms related to _Control using MC methods_ (with ''exploring starts'' or without ''exploring starts'', both on-policy or off-policy).

<u>Questions</u> (**2.5 points**): 
1. Implement the selected algorithm and justify your choice.
2. Comment and justify all the parameters, such as:
- Number of episodes
- Discount factor
- Etc.
3. Implement a function that prints on the screen the optimal policy found for each state (similar to the figure in Section 3.1).
4. Using the trained agent, simulate 100,000 games and calculate the agent's return (total accumulated reward).
5. Additionally, calculate the % of wins, natural wins, losses and draws.

For this part i will implement a On-policy first-visit Monte Carlo Control with $\epsilon$-soft policy, without exploring starts. The justification is that its a simple policy that can be useful for a game like blackjack, with a small state space where we can mantain a table of Q values. And $\epsilon$-soft ensures all actions have a non-zero probability, so we ensure exploration.

I will start with 100.000 episodes, just to check if its working and end with 500.000, the same used for the CliffWalking example, that should be enough. Discount Factor = 1 as in blackjack we care about the only reward at the end of each episode. For epsilon, i will start with a value of Epsilon = 0.1 just to ensure exploration. (maybe to be changed in the future)

In [ ]:
#Alternative version
from collections import defaultdict
import random

# Parameters
n_episodes = 100000
gamma = 1.0
epsilon = 0.1

Q = defaultdict(lambda: np.zeros(env.action_space.n))

def epsilon_greedy_policy(state):
    """Return an action following ε-greedy policy derived from Q."""
    if random.random() < epsilon:
        return env.action_space.sample()
    else:
        return np.argmax(Q[state])

# Monte Carlo Control (On-Policy ε-soft)
returns_sum = defaultdict(lambda: np.zeros(env.action_space.n))
returns_count = defaultdict(lambda: np.zeros(env.action_space.n))

for episode in range(n_episodes):
    episode_memory = []
    state, _ = env.reset()
    done = False

    # Generate one episode
    while not done:
        action = epsilon_greedy_policy(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        episode_memory.append((state, action, reward))
        state = next_state
        done = terminated or truncated

    # Compute returns and update Q
    G = 0
    visited_state_actions = set()
    for state, action, reward in reversed(episode_memory):
        G = gamma * G + reward
        if (state, action) not in visited_state_actions:
            returns_sum[state][action] += G
            returns_count[state][action] += 1
            Q[state][action] = returns_sum[state][action] / returns_count[state][action]
            visited_state_actions.add((state, action))

# Derive the final deterministic (greedy) policy
def optimal_policy(state):
    return np.argmax(Q[state])

print("MC Control training complete.")

#Display policy
def print_policy(Q):
    usable_ace = np.zeros((10, 10), dtype=str)
    no_usable_ace = np.zeros((10, 10), dtype=str)

    for player_sum in range(12, 22):
        for dealer_card in range(1, 11):
            for ace in [True, False]:
                state = (player_sum, dealer_card, ace)
                if state in Q:
                    action = np.argmax(Q[state])
                    symbol = "S" if action == 0 else "H"
                    if ace:
                        usable_ace[player_sum - 12, dealer_card - 1] = symbol
                    else:
                        no_usable_ace[player_sum - 12, dealer_card - 1] = symbol

    print("Policy with usable ace (S=stick, H=hit):")
    print(np.flipud(usable_ace))
    print("\nPolicy without usable ace (S=stick, H=hit):")
    print(np.flipud(no_usable_ace))

print_policy(Q)

#Metrics
episodes_eval = 100000
total_reward = 0
wins = 0
natural_wins = 0
losses = 0
draws = 0

for _ in range(episodes_eval):
    state, info = env.reset()
    terminated = False
    truncated = False
    while not (terminated or truncated):
        action = np.argmax(Q[state])
        state, reward, terminated, truncated, info = env.step(action)

    total_reward += reward
    if reward == 1.0:
        wins += 1
    elif reward == 1.5:
        natural_wins += 1
        wins += 1
    elif reward == 0.0:
        draws += 1
    elif reward == -1.0:
        losses += 1

print(f"Total accumulated reward: {total_reward}")
print(f"Win percentage: {wins / episodes_eval * 100:.2f}%")
print(f"Natural win percentage: {natural_wins / episodes_eval * 100:.2f}%")
print(f"Loss percentage: {losses / episodes_eval * 100:.2f}%")
print(f"Draw percentage: {draws / episodes_eval * 100:.2f}%")


## Part 3. TD learning

The objective of this section is to estimate the optimal policy using TD learning methods. Specifically, you have to implement the **SARSA algorithm**.

<u>Questions</u> (**2.5 points**): 
1. Implement the algorithm.
2. Comment and justify all the parameters.
3. Print on the screen the optimal policy found for each state.
4. Using the trained agent, simulate 100,000 games and calculate the agent's return (total accumulated reward).
5. Additionally, calculate the % of wins, natural wins, losses and draws.

## Part 4. Comparison of the algorithms

In this section, we will make a comparison among the algorithms.

We will compare the performance of the algorithms when changing the number of episodes, the discount factor and the *learning rate* values (in the case of the SARSA method).

For each exercise, the results must be presented and justified.

**Note**: 
- It is recommended to run the simulations multiple times for each exercise, as these are random, and to comment on the most frequent result or the average of these.

### 4.1. Comparison to the optimal policy

The optimal policy for this problem, described by [Sutton & Barto](http://incompleteideas.net/book/the-book-2nd.html) is depicted in the following image:

<img src="./figs/optimal.png" style="width: 800px;" />

<u>Questions</u> (**1 point**): 
- Compare the _optimal_ policies of the naïve, Monte Carlo and SARSA methods to the optimal one provided by Sutton & Barto.
- Comment on the results and justify your answer. 

### 4.2. Influence of the Number of Episodes

Conduct a study by varying the number of episodes in each of the algorithms.

<u>Questions</u> (**1 point**): 
- Train each algorithm multiple times with 100,000, 1,000,000, and 5,000,000 episodes and average the results.
- Indicate how the **number of episodes** influences the convergence of each algorithm by calculating the number of states where the policy differs from the optimal one, as well as the average return obtained after playing 100,000 games following each training.

### 4.3. Influence of the Discount Factor

Conduct a study by varying the *discount factor* in each of the algorithms.

<u>Questions</u> (**1 point**):
- Run the algorithms with *discount factor* = 0.1, 0.5, 0.9 and the rest of the parameters the same as in previous exercises. 
- Describe the changes in the optimal policy, comparing the result obtained with the result of previous exercises (*discount factor* = 1).

### 4.4. Influence of the Learning Rate

Conduct a study by varying the learning rate in the *SARSA* algorithm.

<u>Questions</u> (**1 point**):
- Run the *SARSA* algorithm with the following *learning rate* values: 0.001, 0.01, 0.1, and 0.9.
- Analyze the differences with the results obtained previously in terms of the number of errors relative to the optimal policy and the accumulated reward for every 100,000 episodes played.